In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Bài đầu tiên trong chuỗi bài **Become Data scientist** đó là **Feature selection**, tức là lựa chọn các đặc trưng.

Trong chủ đề này tôi sẽ tìm hiểu các vấn đề chính sau:
* Feature selection là gì. Tại sao phải sử dụng Feature selection?
* Các phương pháp áp dụng cho Feature selection cùng với code example.

# 1. Feature selection là gì?
Là kĩ thuật lựa chọn một tập đặc trưng được coi là có đóng góp cho việc dự đoán hay xây dựng model. 

Ngược lại, chúng ta có thể định nghĩa nó là kĩ thuật loại bỏ các đặc trưng dư thừa, ít đóng góp cho dự đoán hay xây dựng model. 

**Flow nó sẽ như sau:**

Tập feature ban đầu ---Feature Selection--> Tập feature mới ---Modeling--->

Ban đầu từ một tập các feature sau khi áp dụng Feature selection chúng ta sẽ có một tập các feature mới, trong đó tập feature này sẽ nhỏ hơn tập feature ban đầu. Đối với tập feature này chúng sẽ xây dựng và thử nghiệm các mô hình học máy

# Vậy tại sao chúng ta phải sử dụng các kĩ thuật Feature selection?
* Giảm chi phí tính toán về mặt thời gian và bộ nhớ.
    - Đối với bài toán mà dữ liệu nhiều chứa nhiều feature hay nhiều chiều thì sẽ tốn bộ nhớ và thời gian tính toán sẽ lâu hơn vì vậy sẽ rất hạn chế cho chúng ta thử nghiệm với các thí nghiệm khác nhau. Ví dụ thử các mô hình học máy khác nhau, thử tune các hyperpameters ...
    
* Sẽ tốt hơn cho một số thuật toán học máy
    - Với một số thuật toán sẽ thực hiện kém hiệu qua khi số chiều cao. Ví dụ như Linear model, decision tree
    
* Giảm hiện tượng overfit
    - Model sẽ khái quát hóa hơn đối với dữ liệu mới

# Data

Trước khi tìm hiểu các phương pháp Feature Selection chúng ta sẽ import data để phục vụ cho coding example trong mỗi phương pháp Feature selection

Phần này tôi sẽ sử dụng Data từ [Credit Card customers
](https://www.kaggle.com/sakshigoyal7/credit-card-customers). 
Bài toán liên quan đến lĩnh vực ngân hàng, cụ thể chúng ta cần dự đoán khách hàng sẽ rời bỏ các dịch vụ credit card hay không, biến dự đoán là **Attrition_Flag** và các biến feature như: Age, Gender, Education .... 

Đây là bài toán thuộc vào loại classification


In [ ]:
# import các thư viện cần thiết
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)


In [ ]:
# đọc data bằng pandas
df = pd.read_csv("../input/credit-card-customers/BankChurners.csv")

In [ ]:
df.head()

In [ ]:
# Theo như tác giả của data thì chúng ta sẽ loại bỏ 2 cột cuối cùng 
df= df.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis='columns')


Trước khi bắt đầu thì tôi xin bỏ qua phần EDA (Exploratory Data Analysis), phần mà luôn có trong mỗi dự án Data science, chắc chắn nó sẽ trong chuỗi bài **become data scientist** của tôi

Tuy bỏ phần EDA nhưng sẽ tốt hơn khi chúng ta có một cái nhìn tổng thể về data

In [ ]:
df.info()

In [ ]:
df.describe()

Kiểm tra xem dữ liệu có chứa Missing value hay không

In [ ]:
df.isnull().sum()

Theo như kết quả ở trên thì không có hiện tượng Missing value.

Note: Missing value cũng là một vấn đề chúng ta cần nghiên cứu trước khi modeling. Do bài này không bị Missing value nên tôi sẽ viết riêng về chủ đề này

Có điều gì đó không ổn khi các feature ở dạng category, đúng vậy đối với các feature là category thì các thuật toán sao có thể hiểu được. Vậy trước khi áp dụng bất kì thuật toán machine learning nào chúng ta cần biến đổi nó về dạng số, cái mà tôi đang nhắc đến đó là Encoding category. Đối với mỗi loại category sẽ có phương pháp encode khác nhau, đây là một chủ đề mất khá nhiều thời gian nên tôi xin viết riêng. Với bài này tôi sẽ chỉ sử dụng **LabelEncoder** cho tất cả loại category

In [ ]:
from sklearn.preprocessing import LabelEncoder

list_of_cat = ['Attrition_Flag', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category', 'Gender']

for i in list_of_cat:
    df[i] = LabelEncoder().fit_transform(df[i])

In [ ]:
df.head()

In [ ]:
df.shape

Chúng ta có 10127 example và có 19 feature (không đếm CLIENTNUM và Attrition_Flag)

Để tiện cho quá trình đánh giá model tôi sẽ chia dữ liệu thành tập train và test

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['CLIENTNUM', 'Attrition_Flag'])
y = df['Attrition_Flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
y.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
y_train.value_counts()

Như chúng ta thấy có khá nhiều label là 1. Hiện tượng này gọi là data imbalance, tức là mất cân bằng giữa các label. Việc phát hiện và xử lý imbalance đóng vai trong quan trọng trong việc xây dựng model, vậy nên tôi cũng sẽ có riêng 1 bài viết để tìm hiểu về chủ đề này.

# 2. Các phương pháp Feature Selection
Trong phần này tôi sẽ tìm hiểu 2 phương pháp chính đó là:
* 2.1 Unsupervised

* 2.2 Supervised
    * Filter
    * Wrapper

**2.1 Unsupervised**

Phương pháp này chúng ta sẽ không sử dụng đến biến target. Sẽ sử dụng các độ đo trong thống kê như spearman, pearson để loại bỏ các feature dư thừa. Ví dụ khi 2 feature có độ tương quan cao thì chúng ta sẽ chỉ giữ lại 1 trong 2 feature đó.

Code dưới đây tôi sẽ sử dụng 2 độ đo spearman và pearson để tính độ đo liên quan giữa 2 biến là numeric

In [ ]:
# Do không liên quan đến biến dự đoán nên tôi sẽ sử dụng toàn bộ data để tính độ tương quan giữa các feature
corr = X.corr('pearson') # cũng có thể truyền vào spearman
plt.figure(figsize=(14,8))
sns.heatmap(corr, annot=True)
plt.show()

Quan sát đồ thị headmap ở trên chúng ta có thể thấy các cặp feature sau có độ tương quan khá cao:
* **Avg_Open_To_Buy** và **Credit_Limit**, pearson=1
* **Customers_Age** và **Months_on_book**, pearson=0.79

Với mỗi cặp chúng ta sẽ chỉ giữa lại một trong hai. Tôi sẽ bỏ đi **Avg_Open_To_Buy**, **Months_on_book**

In [ ]:
X= X.drop(columns=['Avg_Open_To_Buy', 'Months_on_book'], axis='columns')
X_train = X_train.drop(columns=['Avg_Open_To_Buy', 'Months_on_book'], axis='columns')
X_test = X_test.drop(columns=['Avg_Open_To_Buy', 'Months_on_book'], axis='columns')

# 2.2 Supervised
Như cái tên đã cho chúng ta thấy việc sử dụng label trong việc lựa chọn các feature. Chúng ta sẽ cùng tìm hiểu 2 phương pháp: filter và wrapper


# 2.2.1. Filter
Filter là phương pháp sẽ giúp chúng ta loại bỏ đi các feature kém quan trọng và chỉ giữ lại tập feature quan trọng. Vậy làm thế nào để biết feature nào quan trong và kém quan trọng. Nó sẽ đánh giá mức độ quan trọng của feature dựa vào các kiểm tra thống kế (statistic test) như Chi-Square, ANOVA ..., cụ thể các phương pháp thống kê sẽ scoring mỗi feature bằng cách đánh giá mức độ liên quan với biến target. 

Trong trường hợp này do bài toán là classification nên tôi sẽ sử dụng Chi-Square và ANOVA và Mutual Information. Thực sự trong thời điểm viết chủ đề này tôi cũng chưa thật sự hiểu sâu sắc về phương pháp thống kê như Chi-Square hay ANOVA nên sẽ tìm hiểu và trình bày sau.

**Coding example**

In [ ]:
from sklearn.feature_selection import SelectKBest # 1
from sklearn.feature_selection import chi2 # 2
from sklearn.feature_selection import f_classif # 3
from sklearn.feature_selection import mutual_info_classif # 4
from sklearn.linear_model import LogisticRegression # 5
from sklearn import metrics # 6

* **Giải thích code**

    \# 1 để sử lựa chọn k feature tốt nhất chúng ta sử dung [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)

    \# 2. chi2: tính Chi-Square giữa mỗi feature và biến target

    \# 3. f_classif tính ANOVA giữa mỗi feature và biến target

    \# 4. mutual_info_classif là Mutual Information 

    \# 5. LogisticRegression sẽ sử dụng làm model



In [ ]:
selection_model =  SelectKBest(score_func=chi2, k=10)
selection = selection_model.fit(X_train, y_train)

In [ ]:
selection.get_support()

In [ ]:
selection.scores_

In [ ]:
selection.pvalues_